<a href="https://colab.research.google.com/github/MuhammadHelmyOmar/Hadith/blob/main/bow_featuring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Next

- Preprocessing
  - Removing punctuations, stop words, and diacritics
- Q: How to visualize?
  - box blots for Hadiths lengths
  - cosine similarity between words using embeddings
  - clustering using embeddings (increase word size)
- Increase Dataset size
- Compare important words regardess the model


# Loading Data

In [ ]:
import pandas as pd

In [ ]:
data_url = 'https://raw.githubusercontent.com/KamelGaanoun/MHDetection/main/Corpus/trainFinal.csv'

data = pd.read_csv(data_url, index_col='Unnamed: 0')
print("CSV file loaded successfully!")
# Display the first few rows of the DataFrame
data.head()

CSV file loaded successfully!


,Matan,Degree
19036,لقد رأيتني وأنا ثلث الإسلام،‏.‏,0
17209,هريرة، أن أعرابيا، أتى النبي صلى الله عليه وس...,0
11245,عائشة، قالت كنت أفتل قلائد هدى رسول الله صلى ...,0
2827,قال النبي صلى الله عليه وسلم ‏ ‏ إني فرطكم على...,0
23616,عن النبي صلى الله عليه وسلم قال أقيموا الركوع ...,0


In [ ]:
print(len(data))
print(data['Degree'].value_counts())

21248
Degree
0    19286
1     1962
Name: count, dtype: int64


In [ ]:
# Filter data for each degree
data_degree_0 = data[data['Degree'] == 0]
data_degree_1 = data[data['Degree'] == 1]

# Sample 200 rows from each degree (if available)
# Use .sample(n, random_state) for reproducible sampling
sampled_degree_0 = data_degree_0.sample(n=min(1000, len(data_degree_0)), random_state=42)
sampled_degree_1 = data_degree_1.sample(n=min(1000, len(data_degree_1)), random_state=42)

# Concatenate the sampled dataframes
sampled_data = pd.concat([sampled_degree_0, sampled_degree_1])

# Display the value counts to verify the sampling
print("Value counts of the sampled data:")
display(sampled_data['Degree'].value_counts())

# Display the head of the sampled data
print("\nSampled data head:")
display(sampled_data.head())

Value counts of the sampled data:


,count
Degree,
0,1000
1,1000



Sampled data head:


,Matan,Degree
13645,أنها استعارت من أسماء قلادة فهلكت فأرسل النبي ...,0
16561,يقول كان رسول الله صلى الله عليه وسلم يأتي قبا...,0
13787,هريرة، أن رسول الله صلى الله عليه وسلم قال قلب...,0
4475,أبيه، أن النبي صلى الله عليه وسلم قال لا تزال ...,0
16463,القاسم صلى الله عليه وسلم تسموا باسمي ولا تكنو...,0


In [ ]:
labels = sampled_data['Degree'].values
labels.shape

(2000,)

# Preprocessing

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize
import re
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
vocab = defaultdict(int)

for hadith in sampled_data['Matan']:
    # Use regular expressions to find words composed of alphanumeric characters only
    words = re.findall(r"\b\w+\b", hadith)
    # For each word found, increment its count in the vocab dictionary
    for word in words:
        vocab[word] += 1

# Convert the defaultdict vocab to a regular dictionary for easier handling and sorting
# Sort the dictionary by word frequency in descending order and convert it to a new dictionary
sorted_vocab = dict(sorted(vocab.items(), key=lambda x: x[1], reverse=True))

# Display the sorted vocabulary with each word and its frequency count
display(sorted_vocab)

{'الله': 4267,
 'من': 2227,
 'عليه': 1833,
 'صلى': 1686,
 'قال': 1613,
 'وسلم': 1612,
 'في': 1407,
 'رسول': 1175,
 'أن': 883,
 'على': 852,
 'فقال': 723,
 'يا': 690,
 'لا': 668,
 'ما': 649,
 'النبي': 596,
 'إلى': 595,
 'ثم': 589,
 'عن': 548,
 'له': 470,
 'إلا': 424,
 'كان': 392,
 'بن': 375,
 'إن': 368,
 'ولا': 365,
 'حتى': 344,
 'ذلك': 273,
 'يوم': 270,
 'إذا': 263,
 'هذا': 256,
 'كل': 238,
 'أو': 232,
 'الجنة': 223,
 'لم': 220,
 'فإذا': 185,
 'قد': 181,
 'به': 180,
 'الناس': 179,
 'يقول': 175,
 'أبي': 172,
 'محمد': 171,
 'أهل': 167,
 'وهو': 165,
 'أبو': 164,
 'فلما': 162,
 'عمر': 157,
 'ومن': 152,
 'علي': 150,
 'ابن': 150,
 'جبريل': 147,
 'منه': 145,
 'النار': 140,
 'هو': 140,
 'بين': 139,
 'لي': 137,
 'تعالى': 137,
 'فيه': 135,
 'فإن': 135,
 'عبد': 134,
 'وقال': 133,
 'رجل': 131,
 'القيامة': 128,
 'وما': 128,
 'عند': 128,
 'أنه': 126,
 'قالت': 126,
 'الذي': 125,
 'قلت': 125,
 'بكر': 124,
 'وإن': 122,
 'فقلت': 120,
 'عنه': 114,
 'الصلاة': 111,
 'إليه': 110,
 'رأيت': 108,
 'اللهم': 107,

In [ ]:
len(sorted_vocab)

15987

In [ ]:
# Create a CountVectorizer Object
vectorizer = CountVectorizer()

# Fit and transform the corpus
bow = vectorizer.fit_transform(sampled_data['Matan'])

# Print the length of the generated vocabulary
print("Vocabulary Size:", len(vectorizer.get_feature_names_out()))

# Print the Bag-of-Words matrix
print("BoW Representation:")
print(bow.toarray())
print(bow.shape)

Vocabulary Size: 15975
BoW Representation:
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(2000, 15975)


# Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(bow, labels, test_size=0.2,random_state=109)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1600, 15975), (400, 15975), (1600,), (400,))

# Modeling

In [ ]:
from sklearn.svm import SVC, LinearSVC

In [ ]:
clf = LinearSVC(random_state=109)
clf.fit(X_train, y_train)

LinearSVC(random_state=109)

# Evaluation

In [ ]:
from sklearn import metrics

In [ ]:
y_pred = clf.predict(X_test)

print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
print("Precision:", metrics.precision_score(y_test, y_pred))
print("Recall:", metrics.recall_score(y_test, y_pred))
print("F1 Score:", metrics.f1_score(y_test, y_pred))

Accuracy: 0.895
Precision: 0.8706467661691543
Recall: 0.9162303664921466
F1 Score: 0.8928571428571429


In [ ]:
coefficients = list(clf.coef_[0])
words = list(vectorizer.get_feature_names_out())
maximas = {}
minimas = {}

for i in range(100):
  max_value = max(coefficients)
  min_value = min(coefficients)

  mx = coefficients.index(max_value)
  maximas[words[mx]] = max_value
  coefficients.pop(mx)
  words.pop(mx)

  mn = coefficients.index(min_value)
  minimas[words[mn]] = min_value
  coefficients.pop(mn)

In [ ]:
minimas.keys()

dict_keys(['وسلم', 'أسقطتهن', 'الإسلام', 'اسحقوني', 'دنانير', 'قيوم', 'الخليفة', 'قلبا', 'كمثله', 'التقوا', 'نهدي', 'سواه', 'تطلقوا', 'وادعوا', 'الحبشة', 'المخلوقين', 'قاد', 'به', 'بليل', 'بمشيئته', 'ظله', 'متغيرا', 'يرحمه', 'رشحه', 'وفشا', 'يدع', 'شيطانا', 'فحلفوا', 'ملئ', 'عمرى', 'الناكثون', 'نثل', 'بيت', 'ساقطتها', 'خليتني', 'أمدها', 'شربه', 'عام', 'مصلاه', 'ويلقنه', 'يقدر', 'سليم', 'تواصلت', 'الزناد', 'يقبضها', 'رداءك', 'الخوزية', 'خدامها', 'أليم', 'أبيه', 'يبتغي', 'بذروة', 'دفعتها', 'ذراعيها', 'أولادهم', 'فيها', 'ورجمنا', 'حدث', 'اطووا', 'النبيين', 'الميسرة', 'بكرة', 'يتكففون', 'جأشه', 'أحدكما', 'اثنين', 'أهاويل', 'أذنت', 'فليجعله', 'تشرق', 'والتوراة', 'والذاكرات', 'والشأم', 'والشهوات', 'أوليائي', 'يطعم', 'بنفسه', 'للذئب', 'السرداقات', 'الكبر', 'شك', 'أيديهم', 'تأمر', 'تفعله', 'المتوكلين', 'الميزان', 'الكتابة', 'كالملوك', 'الغيلة', 'وطوله', 'إمرة', 'علينا', 'الزيت', 'مليكة', 'كفارة', 'أبي', 'نهينا', 'الأبوين', 'الحبل', 'يجد'])

In [ ]:
maximas.keys()

dict_keys(['وآله', 'علي', 'عوانة', 'اسلكوا', 'دارهم', 'صلاهما', 'الخط', 'عهودهم', 'الأعجب', 'الحليم', 'النصير', 'يعالج', 'فاضربه', 'الإناء', 'جباههم', 'ترعة', 'يمطر', 'الهندباء', 'جانبه', 'ها', 'راويا', 'نطحوه', 'يؤتى', 'البيع', 'مومنة', 'شوال', 'أتاه', 'يغفر', 'يردفني', 'رشحه', 'مسلمة', 'ألاف', 'الأبيض', 'بأذربيجان', 'الفرقان', 'تحت', 'رب', 'يأمنها', 'أتى', 'كالمجاهد', 'جمعا', 'ضوء', 'قاعدا', 'اليقظان', 'شفاعتي', 'لبناتك', 'والجنة', 'الأوساق', 'أطعمنا', 'طينتهما', 'لفعل', 'هاجر', 'قاتلتني', 'مثل', 'نشر', 'مرض', 'أشجار', 'شمسين', 'زنى', 'ترجع', 'أنبت', 'منامها', 'أيقن', 'أسهم', 'ريحها', 'منبر', 'سليمان', 'المقرئ', 'تطروني', 'يسمعنا', 'إلا', 'الصراخ', 'استرعاهم', 'بأضياف', 'حدثتنا', 'فأخذ', 'فعلمت', 'منهن', 'السمك', 'لعظمتها', 'وانقضت', 'ومحق', 'مهل', 'المنان', 'القيام', 'بطعام', 'الشيب', 'الأقربين', 'اهتديتم', 'بالخسف', 'قالتا', 'بمن', 'نعيما', 'إمرة', 'تكونوا', 'الجعفي', 'قدرتي', 'قضت', 'وأوله', 'أعدت'])

In [ ]:
maximas

{'وآله': np.float64(1.0257957438021206),
 'علي': np.float64(0.5679860436237866),
 'عوانة': np.float64(0.5283315338904847),
 'اسلكوا': np.float64(0.508524598416142),
 'دارهم': np.float64(0.506339798746659),
 'صلاهما': np.float64(0.506339798746659),
 'الخط': np.float64(0.5037906243950401),
 'عهودهم': np.float64(0.5037906243950401),
 'الأعجب': np.float64(0.4587996512820793),
 'الحليم': np.float64(0.4544838815098399),
 'النصير': np.float64(0.4544838815098399),
 'يعالج': np.float64(0.4544838815098399),
 'فاضربه': np.float64(0.4410754785604837),
 'الإناء': np.float64(0.42006309645799206),
 'جباههم': np.float64(0.4093859410123304),
 'ترعة': np.float64(0.3834214464724722),
 'يمطر': np.float64(0.3779928102067536),
 'الهندباء': np.float64(0.36660963951032594),
 'جانبه': np.float64(0.36660963951032594),
 'ها': np.float64(0.36660963951032594),
 'راويا': np.float64(0.35932237316112764),
 'نطحوه': np.float64(0.35932237316112764),
 'يؤتى': np.float64(0.3498774400074779),
 'البيع': np.float64(0.340449